In [7]:
from jobspy import scrape_jobs
import pandas as pd
from datetime import date
import csv

# Your configs here

In [8]:
years_old = 300
result_wanted =30
job_title = '.net'
Locations = {
    "Netherlands": result_wanted,
    "Germany": result_wanted,
    "Belgium": result_wanted,
    "Austria": result_wanted,
    "Australia": result_wanted,
    "Canada": result_wanted,
    "France": result_wanted,
    "Ireland": result_wanted,
    "Italy": result_wanted,
    "Spain": result_wanted,
    "UK": result_wanted
}
job_sefixes = ["relocation", "relocate", "visa sponsorship"]


In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

In [10]:

# BEGIN: Combine all results into a single DataFrame
all_results = pd.DataFrame()

for location, results in Locations.items():
        for suffix in job_sefixes:
            try:
                temp_df = scrape_jobs(
                site_name=[
                        "Glassdoor",
                        "indeed",
                        "LinkedIn",
                        # "zip_recruiter"
                    ],
                    search_term=f"{job_title} {suffix}",
                    location=location,
                    results_wanted=results,
                    country_indeed=location,
                    hours_old=years_old,
                )
            except Exception as e:
                 continue
            all_results = pd.concat([all_results, temp_df], ignore_index=True)

print(f"Found {len(all_results)} jobs")
# print(all_results.head())


2024-10-05 16:51:51,415 - JobSpy - INFO - Indeed search page: 1
2024-10-05 16:51:51,416 - JobSpy - INFO - LinkedIn search page: 1
2024-10-05 16:51:52,101 - JobSpy - INFO - Indeed search page: 2
2024-10-05 16:51:52,137 - JobSpy - INFO - Glassdoor search page: 1
2024-10-05 16:51:52,393 - JobSpy - INFO - Indeed found no jobs on page: 2
2024-10-05 16:51:52,394 - JobSpy - INFO - Indeed finished scraping
2024-10-05 16:51:53,182 - JobSpy - INFO - Glassdoor search page: 2
2024-10-05 16:51:53,679 - JobSpy - INFO - Glassdoor finished scraping
2024-10-05 16:51:56,995 - JobSpy - INFO - LinkedIn search page: 2
2024-10-05 16:52:12,101 - JobSpy - INFO - LinkedIn search page: 3
2024-10-05 16:52:18,402 - JobSpy - INFO - LinkedIn search page: 4
2024-10-05 16:52:19,209 - JobSpy - INFO - Linkedin finished scraping
2024-10-05 16:52:19,270 - JobSpy - INFO - Indeed search page: 1
2024-10-05 16:52:19,271 - JobSpy - INFO - LinkedIn search page: 1
2024-10-05 16:52:19,745 - JobSpy - INFO - Glassdoor search page:

Found 584 jobs


In [11]:
distinct_jobs = all_results.drop_duplicates(subset=['title', 'company'])

first_dict_words = ["c#", " .net ", "asp.net", "dotnet", "dot net", "csharp", "c sharp", "c-sharp",]

second_dict_words = ["reloc", "visa" , "sponsor", "relocate", "relocation"]

def contains_both_word_sets(description, first_words, second_words):
    if pd.isna(description):
        return False
    description_lower = description.lower()
    return any(word in description_lower for word in first_words) and any(word in description_lower for word in second_words)

distinct_jobs = distinct_jobs[distinct_jobs['description'].apply(lambda x: contains_both_word_sets(x, first_dict_words, second_dict_words))]

distinct_jobs = distinct_jobs.sort_values(by='date_posted', ascending=False)
print(f"distintcs are: {len(distinct_jobs)}")

distintcs are: 34


In [12]:
distinct_jobs.to_csv(f"jobs-{date.today()}.csv", quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False) # to_excel
# print(distinct_jobs)